In [11]:
from langchain_community.document_loaders import CSVLoader
from langchain.schema import Document
loader = CSVLoader(file_path="books.csv",csv_args={"delimiter": ";"})

data = loader.load()
print(data)
# Aquí modificamos cada documento para separar contenido y metadatos
new_docs = []
for doc in data:
    md = doc.page_content

    # Construir page_content con descripción y género juntos
    description = md.get("description", "")
    genre = md.get("genre", "")
    page_content = f"Description: {description}\nGenre: {genre}"
    # Construir metadatos excluyendo descripción y género
    metadata = {k: v for k, v in md.items() if k not in ["description", "genre"]}

    # Crear nuevo documento con content y metadatos separados
    new_doc = Document(page_content=page_content, metadata=metadata)
    new_docs.append(new_doc)



[Document(metadata={'source': 'books.csv', 'row': 0}, page_content='isbn: 1909531197\nlanguage_code: eng\naverage_rating: 4.10\ndescription: The story of The Boy in the Striped Pyjamasis very difficult to describe. Usually we give some clues about the book on the cover, but in this case we think that would spoil the reading of the book. We think it is important that you start to read without knowing what it is about.If you do start to read this book, you will go on a journey with a nine,year,old boy called Bruno. And sooner or later you will arrive with Bruno at a fence.We hope you never have to cross such a fence.\nAuthors: John Boyne\nnum_pages: 215\npublication_year: 2014\ntitle: The Boy in the Striped Pyjamas\ngenre: history_biography'), Document(metadata={'source': 'books.csv', 'row': 1}, page_content="isbn: 9782246857\nlanguage_code: fre\naverage_rating: 4.39\ndescription: En 1992, Gabriel, dix ans, vit au Burundi avec son pere francais, entrepreneur, sa mere rwandaise et sa peti

AttributeError: 'str' object has no attribute 'get'

In [23]:
# Leer el CSV con pandas
import pandas as pd
df = pd.read_csv("books.csv", sep=";", encoding="utf-8")
# Lista para almacenar los Document
documents = []
for index, row in df.iterrows():
    # Extraer los campos que van al contenido (embedding)
    description = str(row.get("description", ""))
    genre = str(row.get("genre", ""))
    page_content = f"Description: {description}\nGenre: {genre}"
    # Construir metadata con el resto de columnas
    metadata = {
        "isbn": row.get("isbn"),
        "language_code": row.get("language_code"),
        "average_rating": row.get("average_rating"),
        "Authors": row.get("Authors"),
        "num_pages": row.get("num_pages"),
        "publication_year": row.get("publication_year"),
        "title": row.get("title")
    }
    doc = Document(page_content=page_content, metadata=metadata)
    documents.append(doc)

In [6]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain_community.document_loaders import CSVLoader
from pymilvus import CollectionSchema, FieldSchema, DataType, Collection
from pymilvus import connections, utility
import pandas as pd
import os

In [7]:
    df = pd.read_csv("books.csv", sep=";", encoding="utf-8")
    from langchain_text_splitters import RecursiveCharacterTextSplitter
    # Lista para almacenar los Document
    documents = []
    for index, row in df.iterrows():
        # Extraer los campos que van al contenido (embedding)
        description = str(row.get("description", ""))
        genre = str(row.get("genre", ""))
        page_content = f"Description: {description}\nGenre: {genre}"
        # Construir metadata con el resto de columnas
        metadata = {
            "isbn": row.get("isbn"),
            "language_code": row.get("language_code"),
            "average_rating": row.get("average_rating"),
            "Authors": row.get("Authors"),
            "num_pages": row.get("num_pages"),
            "publication_year": row.get("publication_year"),
            "title": row.get("title")
        }
        doc = Document(page_content=page_content, metadata=metadata)
        documents.append(doc)

    # Dividir los documentos
    text_splitter = RecursiveCharacterTextSplitter(
                chunk_size=1000,
                chunk_overlap=200
            )
    splits = text_splitter.split_documents(documents)
    # Crear embeddings y vectorstore
    embeddings = OllamaEmbeddings(
        model="qwen2.5",
        base_url="http://localhost:11434"
    )
    vector = FAISS.from_documents(splits, embeddings)
    

C:\Users\palom\AppData\Local\Temp\ipykernel_9488\778662523.py:30: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(


In [8]:
    from langchain.embeddings import HuggingFaceEmbeddings
    HOST = "localhost"
    PORT = "19530"
    connections.connect(host=HOST, port=PORT)
    COLLECTION_NAME="Pruebas"
# Extrae los embeddings y documentos de tu FAISS
    vectors = vector.index.reconstruct_n(0, vector.index.ntotal)  # Obtiene todos los vectores
    documents = [doc.page_content for doc in splits]       # Tus textos originales
    metadatas = [doc.metadata for doc in splits]           # Metadatos asociados

    # Crea la colección en Milvus (si no existe)
    if not utility.has_collection(COLLECTION_NAME):
        dim = len(vectors[0])  # Dimensión de tus embeddings (ej: 384, 768, etc.)
        
        fields = [
            FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True),
            FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=dim),
            FieldSchema(name="text", dtype=DataType.VARCHAR, max_length=65535),
            FieldSchema(name="metadata", dtype=DataType.JSON)
        ]
        schema = CollectionSchema(fields, description="Índice de embeddings")
        collection = Collection(name=COLLECTION_NAME, schema=schema)
        
        # Crea un índice HNSW para búsquedas eficientes
        index_params = {
            "metric_type": "L2",  # Distancia euclidiana (usa "IP" para producto interno)
            "index_type": "HNSW",
            "params": {"M": 16, "efConstruction": 200}
        }
        collection.create_index(field_name="embedding", index_params=index_params)

    # Guarda los datos en Milvus
    data = [
        vectors,                    # Embeddings
        documents,                  # Textos originales
        metadatas                   # Metadatos (opcional)
    ]

    # Inserta los datos
    collection = Collection(COLLECTION_NAME)
    collection.insert(data)
    collection.flush()
    collection.load()

In [ ]:
print("Colecciones existentes:", utility.list_collections())
collection.flush()
collection.load()


Colecciones existentes: ['Pruebas']


In [43]:
print("Campos:", collection.schema.fields)

Campos: [{'name': 'id', 'description': '', 'type': <DataType.INT64: 5>, 'is_primary': True, 'auto_id': True}, {'name': 'embedding', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 3584}}, {'name': 'text', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 65535}}, {'name': 'metadata', 'description': '', 'type': <DataType.JSON: 23>}]


In [44]:
collection.load()
results = collection.query(
    expr="",  # Sin filtro
    output_fields=["text", "metadata"],
    limit=5
)
for r in results:
    print(r)

{'metadata': {'isbn': 1909531197, 'language_code': 'eng', 'average_rating': 4.1, 'Authors': 'John Boyne', 'num_pages': 215, 'publication_year': 2014, 'title': 'The Boy in the Striped Pyjamas'}, 'id': 458622025556822550, 'text': 'Description: The story of The Boy in the Striped Pyjamasis very difficult to describe. Usually we give some clues about the book on the cover, but in this case we think that would spoil the reading of the book. We think it is important that you start to read without knowing what it is about.If you do start to read this book, you will go on a journey with a nine,year,old boy called Bruno. And sooner or later you will arrive with Bruno at a fence.We hope you never have to cross such a fence.\nGenre: history_biography'}
{'metadata': {'isbn': 9782246857, 'language_code': 'fre', 'average_rating': 4.39, 'Authors': 'Gael Faye', 'num_pages': 217, 'publication_year': 2016, 'title': 'Petit Pays'}, 'id': 458622025556822551, 'text': "Description: En 1992, Gabriel, dix ans,

In [ ]:
from langchain_community.vectorstores import Milvus
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")   
vectorstore = Milvus(
            collection_name="Pruebas",
            embedding_function=embeddings,
            vector_field="embeddings",
            text_field="text",
            connection_args={
                "host": "localhost",
                "port": 19530

                
            }
        )


    

docs = vectorstore.similarity_search("science fiction book about artificial intelligence", k=1)
print(docs)

TypeError: Milvus.__init__() got an unexpected keyword argument 'embedding_field'

In [41]:
# Frase de consulta
model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
query = "science fiction book about artificial intelligence"
query_vector = model.encode(query).tolist()

# Lanza la búsqueda en Milvus
results = collection.search(
    data=[query_vector],            # lista de vectores a buscar
    anns_field="embedding",         # nombre del campo con los vectores
    param={"metric_type": "L2", "params": {"ef": 128}},
    limit=5,
    output_fields=["text", "metadata"]
)

# Muestra resultados
for hits in results:
    for hit in hits:
        print("Score:", hit.distance)
        print("Texto:", hit.entity.get("text"))
        print("Metadata:", hit.entity.get("metadata"))
        print("-" * 40)

C:\Users\palom\AppData\Local\Temp\ipykernel_19348\2122410605.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
c:\Users\palom\.conda\envs\UNIR_Practicas\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


AttributeError: 'HuggingFaceEmbeddings' object has no attribute 'encode'